In [ ]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [ ]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [ ]:
def model(t_u, w1,w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [ ]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [ ]:
w1 = torch.ones(())
w2 = torch.ones(())
b = torch.zeros(())

t_p = model(t_u, w1,w2, b)
t_p

In [ ]:
loss = loss_fn(t_p, t_c)
loss

In [ ]:
x = torch.ones(())
y = torch.ones(3,1)
z = torch.ones(1,3)
a = torch.ones(2, 1, 1)
print(f"shapes: x: {x.shape}, y: {y.shape}")
print(f"        z: {z.shape}, a: {a.shape}")
print("x * y:", (x * y).shape)
print("y * z:", (y * z).shape)
print("y * z * a:", (y * z * a).shape)

In [ ]:
delta = 0.1

loss_rate_of_change_w1 = \
    (loss_fn(model(t_u, w1 + delta, w2, b), t_c) - 
     loss_fn(model(t_u, w1 - delta, w2, b), t_c)) / (2.0 * delta)
    
loss_rate_of_change_w2 = \
    (loss_fn(model(t_u, w1, w2+delta, b), t_c) - 
     loss_fn(model(t_u, w1, w2-delta, b), t_c)) / (2.0 * delta)

In [ ]:
learning_rate = 1e-2

w1 = w1 - learning_rate * loss_rate_of_change_w1
w2 = w2 - learning_rate * loss_rate_of_change_w2

In [ ]:
loss_rate_of_change_b = \
    (loss_fn(model(t_u, w1,w2, b + delta), t_c) - 
     loss_fn(model(t_u, w1,w2, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [ ]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)  # <1>
    return dsq_diffs

In [ ]:
def dmodel_dw(t_u, w1,w2, b):
    return t_u

In [ ]:
def dmodel_db(t_u, w1,w2, b):
    return 1.0

In [ ]:
def grad_fn(t_u, t_c, t_p, w1,w2, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw1 = dloss_dtp * dmodel_dw(t_u, w1,w2, b)
    dloss_dw2 = dloss_dtp * dmodel_dw(t_u, w1,w2, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w1,w2, b)
    return torch.stack([dloss_dw1.sum(),dloss_dw2.sum(), dloss_db.sum()])  # <1>

In [ ]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w1,w2, b = params

        t_p = model(t_u, w1,w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1,w2, b)  # <2>

        params = params - learning_rate * grad

        print('Epoch %d, Loss %f' % (epoch, float(loss))) # <3>
            
    return params

In [ ]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c,
                  print_params=True):
    for epoch in range(1, n_epochs + 1):
        w1,w2, b = params
        t_p = model(t_u, w1,w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1,w2, b)  # <2>

        params = params - learning_rate * grad

        # report loss after every 500 epochs
        if epoch % 500 == 0:  # <3>
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            if print_params:
                print('    Params:', params)
                print('    Grad:  ', grad)
        if epoch in {4, 12, 101}:
            print('...')
        
            
        if not torch.isfinite(loss).all():
            break  # <3>
            
    return params

In [ ]:
t_un = 0.1 * t_u

In [ ]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-4, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_un, 
    t_c = t_c,
    print_params = True)

params

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

t_p = model(t_un, *params)  # <1>

fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy()) # <2>
plt.plot(t_u.numpy(), t_c.numpy(), '+')
plt.savefig("temp_unknown_plot_e4.png", format="png")  # bookskip

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

fig = plt.figure(dpi=600)
plt.xlabel("Measurement")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')

plt.savefig("temp_data_plot_e4.png", format="png")

In [ ]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-3, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_un, 
    t_c = t_c,
    print_params = True)

params

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

t_p = model(t_un, *params)  # <1>

fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy()) # <2>
plt.plot(t_u.numpy(), t_c.numpy(), '+')
plt.savefig("temp_unknown_ploe3.png", format="png")  # bookskip

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

fig = plt.figure(dpi=600)
plt.xlabel("Measurement")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')

plt.savefig("temp_data_plot_e3.png", format="png")

In [ ]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_un, 
    t_c = t_c,
    print_params = True)

params

%matplotlib inline
from matplotlib import pyplot as plt

t_p = model(t_un, *params)  # <1>

fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy()) # <2>
plt.plot(t_u.numpy(), t_c.numpy(), '+')
plt.savefig("temp_unknown_plot_e2.png", format="png")  # bookskip



In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(dpi=600)
plt.xlabel("Measurement")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')

plt.savefig("temp_data_plot_e2.png", format="png")

In [ ]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-1, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_un, 
    t_c = t_c,
    print_params = True)

params
%matplotlib inline
from matplotlib import pyplot as plt

t_p = model(t_un, *params)  # <1>

fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy()) # <2>
plt.plot(t_u.numpy(), t_c.numpy(), '+')
plt.savefig("temp_unknown_plot_e1.png", format="png")  # bookskip


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

fig = plt.figure(dpi=600)
plt.xlabel("Measurement")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')

plt.savefig("temp_data_plot_e1.png", format="png")